In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import block, helper, network

### Exercise 1
Parse this message
```
f9beb4d976657261636b000000000000000000005df6e0e2
```


In [ ]:
# Exercise 1

from helper import little_endian_to_int
msg = bytes.fromhex('f9beb4d976657261636b000000000000000000005df6e0e2')
# first 4 are network magic

# next 12 are command

# next 4 are payload length

# next 4 are checksum

# rest is payload

# print the command


### Exercise 2




#### Make [this test](/edit/session7/network.py) pass: `network.py:NetworkEnvelopeTest:test_parse`

In [ ]:
# Exercise 2

reload(network)
run(network.NetworkEnvelopeTest('test_parse'))

### Exercise 3




#### Make [this test](/edit/session7/network.py) pass: `network.py:NetworkEnvelopeTest:test_serialize`

In [ ]:
# Exercise 3

reload(network)
run(network.NetworkEnvelopeTest('test_serialize'))

### Exercise 4




#### Make [this test](/edit/session7/network.py) pass: `network.py:GetHeadersMessageTest:test_serialize`

In [ ]:
# Exercise 4

reload(network)
run(network.GetHeadersMessageTest('test_serialize'))

### Exercise 5




#### Make [this test](/edit/session7/network.py) pass: `network.py:HeadersMessageTest:test_parse`

In [ ]:
# Exercise 5

reload(network)
run(network.HeadersMessageTest('test_parse'))

In [ ]:
# Handshake Example
from network import SimpleNode, VersionMessage, VerAckMessage
node = SimpleNode('signet.programmingbitcoin.com', network="signet")
version = VersionMessage()
node.send(version)
print(node.wait_for(VerAckMessage).command)

### Exercise 6




#### Make [this test](/edit/session7/network.py) pass: `network.py:SimpleNodeTest:test_handshake`

In [ ]:
# Exercise 6

reload(network)
run(network.SimpleNodeTest('test_handshake'))

In [ ]:
# Block Header Download Example
from block import GENESIS_BLOCKS
from helper import target_to_bits
from network import GetHeadersMessage, HeadersMessage, SimpleNode
net = "mainnet"
node = SimpleNode('mainnet.programmingbitcoin.com', network=net)
node.handshake()
last_block = GENESIS_BLOCKS[net]
epoch_start_block = GENESIS_BLOCKS[net]
current_height = 1
for _ in range(20):
    getheaders = GetHeadersMessage(start_block=last_block.hash())
    node.send(getheaders)
    headers = node.wait_for(HeadersMessage)
    for header in headers:
        if not header.check_pow():
            raise RuntimeError(f'bad proof of work at block {count}')
        if last_block != GENESIS_BLOCKS[net]:
            if header.prev_block != last_block.hash():
                raise RuntimeError(f'discontinuous block at {count}')
            if current_height % 2016 == 0:
                expected_bits = last_block.new_bits(epoch_start_block)
                if header.bits != expected_bits:
                    raise RuntimeError(f'bits are off {header.bits.hex()} vs {expected_bits.hex()}')
                epoch_start_block = header
                print(header.id())
        last_block = header
        current_height += 1

### Exercise 7
Download the first 40,000 blocks for testnet and validate them.


In [ ]:
# Exercise 7

from network import SimpleNode, GetHeadersMessage, HeadersMessage
from block import GENESIS_BLOCKS
# connect to testnet.programmingbitcoin.com with network
net = "testnet"

# handshake

# set the last block hash to the GENESIS_BLOCKS[net]

# set the first block of the epoch to the genesis block

# set the current height to 1

# loop until we we get 40,000 blocks

    # create a GetHeadersMessage starting from the last block we have

    # send the getheaders message

    # wait for the HeadersMessage in response

    # loop through the headers from the headers message

        # check the proof of work


        # only check if the current hash isn't the first one

            # the prev_block of the current block should be the last block


            # when it's a multiple of 2016

                # set the expected bits using the new_bits method using the last block

                # check that the bits are what we expect


                # set the epoch start block to the current one



        # increment the current_height

        # set the last_block to the current header's hash


In [ ]:
# Merkle Parent Example
from helper import hash256
tx_hash0 = bytes.fromhex('c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5')
tx_hash1 = bytes.fromhex('c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5')
parent = hash256(tx_hash0+tx_hash1)
print(parent.hex())

### Exercise 8
Calculate the Merkle parent of these hashes:
```
f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0
3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181
```


In [ ]:
# Exercise 8

from helper import hash256
hex_hash1 = 'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0'
hex_hash2 = '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181'
# bytes.fromhex to get the bin hashes


# hash256 the combination

# hex() to see the result


### Exercise 9




#### Make [this test](/edit/session7/helper.py) pass: `helper.py:HelperTest:test_merkle_parent`

In [ ]:
# Exercise 9

reload(helper)
run(helper.HelperTest('test_merkle_parent'))

In [ ]:
# Merkle Parent Level Example
from helper import merkle_parent
hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
]
hashes = [bytes.fromhex(x) for x in hex_hashes]
if len(hashes) % 2 == 1:
    hashes.append(hashes[-1])
parent_level = []
for i in range(0, len(hex_hashes), 2):
    parent = merkle_parent(hashes[i], hashes[i+1])
    print(parent.hex())
    parent_level.append(parent)

### Exercise 10
Calculate the next Merkle Parent Level given these hashes
```
8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd
7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800
ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7
68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069
43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27
```


In [ ]:
# Exercise 10

from helper import merkle_parent
hex_hashes = [
    '8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd',
    '7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800',
    'ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7',
    '68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069',
    '43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27',
]
# bytes.fromhex to get all the hashes in binary

# if the number of hashes is odd, duplicate the last one


# initialize parent level

# skip by two: use range(0, len(hashes), 2)

    # calculate merkle_parent of i and i+1 hashes

    # print the hash's hex

    # add parent to parent level


### Exercise 11




#### Make [this test](/edit/session7/helper.py) pass: `helper.py:HelperTest:test_merkle_parent_level`

In [ ]:
# Exercise 11

reload(helper)
run(helper.HelperTest('test_merkle_parent_level'))

In [ ]:
# Merkle Root Example
from helper import merkle_parent_level
hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
    '7d37b3d54fa6a64869084bfd2e831309118b9e833610e6228adacdbd1b4ba161',
    '8118a77e542892fe15ae3fc771a4abfd2f5d5d5997544c3487ac36b5c85170fc',
    'dff6879848c2c9b62fe652720b8df5272093acfaa45a43cdb3696fe2466a3877',
    'b825c0745f46ac58f7d3759e6dc535a1fec7820377f24d4c2c6ad2cc55c0cb59',
    '95513952a04bd8992721e9b7e2937f1c04ba31e0469fbe615a78197f68f52b7c',
    '2e6d722e5e4dbdf2447ddecc9f7dabb8e299bae921c99ad5b0184cd9eb8e5908',
    'b13a750047bc0bdceb2473e5fe488c2596d7a7124b4e716fdd29b046ef99bbf0',
]
current_level = [bytes.fromhex(x) for x in hex_hashes]
while len(current_level) > 1:
    current_level = merkle_parent_level(current_level)
print(current_level[0].hex())

### Exercise 12
Calculate the Merkle Root given these hashes
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```


In [ ]:
# Exercise 12

from helper import merkle_parent_level
hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]
# bytes.fromhex to get all the hashes in binary

# initialize current level to be the hashes

# loop until current_level has only 1 element

    # make the current level the parent level

# print the root's hex


### Exercise 13




#### Make [this test](/edit/session7/helper.py) pass: `helper.py:HelperTest:test_merkle_root`

In [ ]:
# Exercise 13

reload(helper)
run(helper.HelperTest('test_merkle_root'))

In [ ]:
# Block Merkle Root Example
from helper import merkle_root
tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
]
current_level = [bytes.fromhex(x)[::-1] for x in tx_hex_hashes]
print(merkle_root(current_level)[::-1].hex())

### Exercise 14
Validate the merkle root for this block from Testnet:
Block Hash:
```
0000000000000451fa80fcdb243b84c35eaae215a85a8faa880559e8239e6f20
```

Transaction Hashes:
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```


In [ ]:
# Exercise 14

from helper import merkle_root
want = '4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301'
tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]
# bytes.fromhex and reverse ([::-1]) to get all the hashes in binary

# get the merkle root

# see if the reversed root is the same as the wanted root


### Exercise 15




#### Make [this test](/edit/session7/block.py) pass: `block.py:BlockTest:test_validate_merkle_root`

In [ ]:
# Exercise 15

reload(block)
run(block.BlockTest('test_validate_merkle_root'))

### Exercise 16
Validate the merkle root for this block from Testnet via network protocol:
Block Hash:
```
0000000000044b01a9440b34f582fe171c7b8642fedd0ebfccf8fdf6a1810900
```


In [ ]:
# Exercise 16

from network import SimpleNode, GetDataMessage, BLOCK_DATA_TYPE
from block import Block
block_hex = '0000000000044b01a9440b34f582fe171c7b8642fedd0ebfccf8fdf6a1810900'
block_hash = bytes.fromhex(block_hex)
# connect to testnet.programmingbitcoin.com on testnet

# handshake

# create a GetDataMessage

# add_data on the message (BLOCK_DATA_TYPE, block_hash)

# send the getdata message

# wait for the block message in response

# check the proof of work


# validate the tx_hashes


# print the merkle root hex
